<a href="https://colab.research.google.com/github/isilberfin/Data-Augmentation-NLP/blob/main/distributed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

#nlpaug

In [ ]:
!pip install torch>=1.6.0 transformers>=4.0.0 sentencepiece
!pip install nltk>=3.4.5
!pip install numpy requests nlpaug

In [3]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action

In [4]:
import pandas as pd
import sklearn
from sklearn.utils import shuffle
data = pd.read_csv("amazontrain.csv")

In [6]:
grouped = data.groupby(data["Y"])
one_labeled_texts = grouped.get_group(1)
zero_labeled_texts = grouped.get_group(0)

one_labeled_texts = one_labeled_texts.reset_index(drop= True)
zero_labeled_texts = zero_labeled_texts.reset_index(drop= True)


seperated_data_zero = []
seperated_data_one = []

data = zero_labeled_texts["text"]

context = ""
for j in range(len(data)):
  seperated_data_zero.append(data[j])


data = one_labeled_texts["text"]

context = ""
for j in range(len(data)):
  seperated_data_one.append(data[j])


In [8]:
len(seperated_data_zero), len(seperated_data_one)

(400, 400)

In [9]:
aug = nac.RandomCharAug(action="delete")

In [10]:
augmented_0_data=[]

for s in range(len(seperated_data_zero)):
  augmented_0_data.append(aug.augment(seperated_data_zero[s],n=1))

In [11]:
augmented_1_data=[]

for s in range(len(seperated_data_one)):
  augmented_1_data.append(aug.augment(seperated_data_one[s],n=1))

In [12]:
random_delete_0 = pd.DataFrame(augmented_0_data, columns = ['text'])
random_delete_1 = pd.DataFrame(augmented_1_data, columns = ['text'])

In [13]:
random_delete_1['Y'] = 1
random_delete_0['Y'] = 0

In [14]:
random_delete_0.to_csv('distributed_rd_0.csv',index=False)
random_delete_1.to_csv('distributed_rd_1.csv',index=False)

In [16]:
augmented_data = random_delete_0.append(random_delete_1)

In [17]:
augmented_data = augmented_data.reset_index(drop=True)

In [19]:
augmented_data = shuffle(augmented_data)
augmented_data = augmented_data.reset_index(drop = True)

In [25]:
train_orig = pd.read_csv("amazontrain.csv")

In [26]:
aug_orig = train_orig.append(augmented_data)
aug_orig = aug_orig.reset_index(drop=True)

aug_orig = shuffle(aug_orig)
aug_orig = aug_orig.reset_index(drop = True)

In [28]:
aug_orig.head(4)

,Y,text
0,0,"I was sitting in my hice, wt the crde on my bl..."
1,0,"Then I echged for the me hne, even tt had the ..."
2,1,cool phone.
3,0,Not od enough and doesn ' t tr on lk it sold.


In [29]:
aug_orig.shape

(1600, 2)

#training

In [30]:
import pandas as pd
test = pd.read_csv("amazontest.csv")

In [35]:
from sklearn.model_selection import train_test_split
test_sentences, val_sentences, test_labels, val_labels = train_test_split(test["text"].to_numpy(),
                                                                            test["Y"].to_numpy(),
                                                                            test_size=0.9,
                                                                           random_state=None) 

train_sentences = aug_orig['text'].to_numpy()
train_labels = aug_orig['Y'].to_numpy()

In [36]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(1600, 1600, 180, 180)

In [40]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace", 
                                    output_mode="tf_idf",
                                    output_sequence_length=None)

In [75]:
def vectorize(data):
  return text_vectorizer.adapt(data["text"]), data["Y"]

In [ ]:
aug_orig

In [139]:
def create_dataset(train_data, batch_size):
  train_data = tf.data.experimental.make_csv_dataset(train_data, batch_size=BATCH_SIZE)
  train_data = train_data.map(vectorize,
                              num_parallel_calls = tf.data.experimental.AUTOTUNE)
  train_data = train_data.shuffle(1600)
  train_data = train_data.batch(batch_size)
  train_data = train_data.prefetch(tf.data.experimental.AUTOTUNE)
  return train_data

In [140]:
from tensorflow.keras import layers
max_vocab_length = 10000 
max_length = 15 
embedding = layers.Embedding(input_dim=max_vocab_length, 
                             output_dim=128,
                             embeddings_initializer="uniform", 
                             input_length=max_length, 
                             name="embedding_1") 

In [141]:
def create_model():
  inputs = layers.Input(shape=(max_length,))
  x = embedding(inputs)
  x = layers.Dropout(0.2)(x)
  x = layers.GlobalAveragePooling1D()(x) 
  x = layers.Dropout(0.2)(x)
  outputs = layers.Dense(1, activation="sigmoid")(x) 
  model = tf.keras.Model(inputs, outputs, name="model_1_dense") 
  return model

In [142]:
model = create_model()
model.compile(
    loss = "sparse_categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(0.0001),
    metrics = ["Accuracy"]
)

In [143]:
BATCH_SIZE = 32

In [146]:
train_data = create_dataset('aug_orig.csv', BATCH_SIZE)

RuntimeError: ignored